In [1]:
import pandas as pd
import numpy as np

In [2]:
maids, email, phone = pd.read_csv("maids.csv"), pd.read_csv("phone.csv"), pd.read_csv("email.csv")

In [3]:
f1, f2, f3, f4 = pd.read_csv("final_table/f1.csv"), pd.read_csv("final_table/f2.csv"), pd.read_csv("final_table/f3.csv"), pd.read_csv("final_table/f4.csv")

In [4]:
f1.shape[0] + f2.shape[0] + f3.shape[0] + f4.shape[0] 

1000000

In [5]:
for df in [f1, f2, f3, f4]:
    df.fillna('-', inplace=True)
    df['id'] = df.fn + df.ln + df.dob.astype(str) + df.st + df.zip.astype(str)

In [6]:
for df in [maids, email, phone]:
    df.fillna('-', inplace=True)
    df['id'] = df.vb_tsmart_first_name + df.vb_tsmart_last_name + df.vb_voterbase_dob.astype(str) + \
    df.vb_tsmart_state + df.vb_tsmart_zip.astype(str)

In [12]:
dfs = [f1, f2, f3, f4]
for i in range(0, len(dfs)):
    df = dfs[i]
    # first create phone sub
    phone_sub = df.merge(phone, on='id', how='left')
    cols = ['Unnamed: 0', 'fn', 'ln', 'vb_voterbase_phone', 'dob', 'age', 'gen', 'ct', 'st',
       'zip', 'vb_email', 'country', 'id']
    # if voterbase phone is null, use wireless phone instead since only add rows but not columns
    phone_sub.loc[phone_sub.vb_voterbase_phone == '-', 
                  'vb_voterbase_phone'] = phone_sub.vb_voterbase_phone_wireless
    # get all records from phone table after merge
    # rename for appending
    # this is all the new phone, email from phone dataset
    phone_sub_app = phone_sub[cols].rename(columns={'vb_voterbase_phone' : 'phone',
                                   'vb_email' : 'email'})
    
    # second create maid sub
    maid_sub = df.merge(maids, on='id', how='left')
    cols = ['Unnamed: 0', 'fn', 'ln', 'vb_voterbase_phone', 'dob', 'age', 'gen', 'ct', 'st',
       'zip', 'vb_email', 'country', 'id']
    # if voterbase phone is null, use wireless phone instead since only add rows but not columns
    maid_sub.loc[maid_sub.vb_voterbase_phone == '-', 
                  'vb_voterbase_phone'] = maid_sub.vb_voterbase_phone_wireless
    # get all records from maid table after merge
    # rename for appending
    # this is all the new phone, email from maid dataset
    maid_sub_app = maid_sub[cols].rename(columns={'vb_voterbase_phone' : 'phone',
                                   'vb_email' : 'email'})
    
    
    # appending
    maid_subf, phone_subf = maid_sub_app[df.columns], phone_sub_app[df.columns]
    df_enriched = df.append(maid_subf)
    df_enriched = df_enriched.append(phone_subf)
    cols = list(df_enriched.columns) + ['native_maid']
    # add maid value to df 
    df_enriched_maid = df_enriched.merge(maids, on='id', how='left')[cols]
    df_enriched_maid = df_enriched_maid.rename(columns={'native_maid' : 'madid'})
    df_enriched_maid_dedup = df_enriched_maid.drop_duplicates()
    print("Shape after de-duplicates for: " + str(i + 1), df_enriched_maid_dedup.shape[0])
    df_enriched_maid_dedup.to_csv("final_table/f" + str(i + 1) + '_enriched.csv')

Shape after de-duplicates for: 1 581797
Shape after de-duplicates for: 2 467009
Shape after de-duplicates for: 3 105164
Shape after de-duplicates for: 4 380116
